In [2]:
import pandas as pd
import sqlite3

In [3]:
pd.options.display.max_columns = None

In [4]:
conn = sqlite3.connect('sakila.db')

In [5]:
cursor = conn.cursor()
cursor.execute('''
SELECT * FROM sqlite_master WHERE type = 'table'
''')
data = cursor.fetchall()
columns = [col[0] for col in cursor.description]
cursor.close()

In [6]:
data_df = pd.DataFrame(data, columns=columns)
data_df

,type,name,tbl_name,rootpage,sql
0,table,actor,actor,2,"CREATE TABLE ""actor"" (\n""index"" INTEGER,\n ""a..."
1,table,store,store,7,"CREATE TABLE ""store"" (\n""index"" INTEGER,\n ""s..."
2,table,address,address,9,"CREATE TABLE ""address"" (\n""index"" INTEGER,\n ..."
3,table,category,category,26,"CREATE TABLE ""category"" (\n""index"" INTEGER,\n ..."
4,table,city,city,28,"CREATE TABLE ""city"" (\n""index"" INTEGER,\n ""ci..."
5,table,country,country,39,"CREATE TABLE ""country"" (\n""index"" INTEGER,\n ..."
6,table,customer,customer,43,"CREATE TABLE ""customer"" (\n""index"" INTEGER,\n ..."
7,table,film_actor,film_actor,63,"CREATE TABLE ""film_actor"" (\n""index"" INTEGER,\..."
8,table,film_category,film_category,126,"CREATE TABLE ""film_category"" (\n""index"" INTEGE..."
9,table,inventory,inventory,140,"CREATE TABLE ""inventory"" (\n""index"" INTEGER,\n..."


In [7]:
pd.read_sql_query(sql = '''
SELECT * FROM actor;
''', con = conn)

,index,actor_id,first_name,last_name,last_update
0,0,1,Penelope,Guiness,2013-05-26 14:47:57.620000
1,1,2,Nick,Wahlberg,2013-05-26 14:47:57.620000
2,2,3,Ed,Chase,2013-05-26 14:47:57.620000
3,3,4,Jennifer,Davis,2013-05-26 14:47:57.620000
4,4,5,Johnny,Lollobrigida,2013-05-26 14:47:57.620000
...,...,...,...,...,...
195,195,196,Bela,Walken,2013-05-26 14:47:57.620000
196,196,197,Reese,West,2013-05-26 14:47:57.620000
197,197,198,Mary,Keitel,2013-05-26 14:47:57.620000
198,198,199,Julia,Fawcett,2013-05-26 14:47:57.620000


Question 1:

Which categories yield most amount of money? Include TOTAL row.

In [8]:
pd.read_sql_query(sql = '''
WITH t1 AS (SELECT c.name AS name, SUM(p.amount) AS total_amount, COUNT(r.rental_id) as number_of_rentals
FROM film AS f
LEFt JOIN film_category AS fc
ON f.film_id = fc.film_id
LEFt JOIN category AS c
ON fc.category_id = c.category_id
JOIN inventory AS i
ON i.film_id = f.film_id
JOIN rental AS r
ON i.inventory_id = r.inventory_id
JOIN payment AS p
ON p.rental_id = r.rental_id
GROUP BY 1
ORDER BY 2 DESC)

SELECT *
FROM t1
UNION ALL
SELECT 'TOTAL', SUM(total_amount), SUM(number_of_rentals)
FROM t1;
''', con = conn)

,name,total_amount,number_of_rentals
0,Sports,4892.19,1081
1,Sci-Fi,4336.01,998
2,Animation,4245.31,1065
3,Drama,4118.46,953
4,Comedy,4002.48,851
5,New,3966.38,864
6,Action,3951.84,1013
7,Foreign,3934.47,953
8,Games,3922.18,884
9,Family,3830.15,988


Solution:

WITH t1 AS (SELECT c.name AS name, SUM(p.amount) AS total_amount, COUNT(r.rental_id) as number_of_rentals
FROM film AS f
LEFt JOIN film_category AS fc
ON f.film_id = fc.film_id
LEFt JOIN category AS c
ON fc.category_id = c.category_id
JOIN inventory AS i
ON i.film_id = f.film_id
JOIN rental AS r
ON i.inventory_id = r.inventory_id
JOIN payment AS p
ON p.rental_id = r.rental_id
GROUP BY 1
ORDER BY 2 DESC)

SELECT *
FROM t1
UNION ALL
SELECT 'TOTAL', SUM(total_amount), SUM(number_of_rentals)
FROM t1;

In [25]:
# check sum

pd.read_sql_query(sql = '''
SELECT c.name AS name, SUM(p.amount) AS total_amount, COUNT(r.rental_id) as number_of_rentals
FROM film AS f
LEFt JOIN film_category AS fc
ON f.film_id = fc.film_id
LEFt JOIN category AS c
ON fc.category_id = c.category_id
JOIN inventory AS i
ON i.film_id = f.film_id
JOIN rental AS r
ON i.inventory_id = r.inventory_id
JOIN payment AS p
ON p.rental_id = r.rental_id
''', con = conn)

,name,total_amount,number_of_rentals
0,Music,61312.04,14596


In [22]:
# check sum

pd.read_sql_query(sql = '''
SELECT SUM(p.amount) AS total_amount
FROM payment AS p
''', con = conn)

,total_amount
0,61312.04


In [26]:
# check sum

pd.read_sql_query(sql = '''
SELECT COUNT(r.rental_id) as number_of_rentals
FROM rental AS r
JOIN payment AS p
ON p.rental_id = r.rental_id
''', con = conn)

,number_of_rentals
0,14596
